In [1]:
import torch
import time
from snn_delays.snn_refactored import SNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.train_utils_refact_minimal import train, get_device, propagate_batch_simple
from snn_delays.utils.test_behavior import tb_minimal

'''
SHD dataset as in ablation study
'''

device = get_device()

# for reproducibility
torch.manual_seed(10)

dataset = 'ibm_gestures'
num_steps = 50
batch_size = 32

# DATASET
DL = DatasetLoader(dataset=dataset,
                  caching='gpu',
                  num_workers=0,
                  batch_size=batch_size,
                  total_time=num_steps,
                  sensor_size_to=32,
                  crop_to=1e6)
train_loader, test_loader, dataset_dict = DL.get_dataloaders()

c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on: cuda:0
[CropTime(min=0, max=1000000.0), Downsample(time_factor=1, spatial_factor=0.25, sensor_size=None, target_size=None), ToFrame(sensor_size=(32, 32, 2), time_window=None, event_count=None, n_time_bins=50, n_event_bins=None, overlap=0, include_incomplete=False)]
Initializing GPU cache for 1077 samples...
Cache initialized in 58.64 seconds
Initializing GPU cache for 264 samples...
Cache initialized in 16.32 seconds


In [2]:
for img, label in test_loader:
    print('img shape:', img.shape)
    print('label shape:', label.shape)
    break

img shape: torch.Size([32, 50, 2, 32, 32])
label shape: torch.Size([32, 11])


In [ ]:
structure = (64, 3, 'f')
extra_kwargs = {}

snn = SNN(dataset_dict=dataset_dict, structure=structure, tau_m='log-normal', win=num_steps, loss_fn='mem_sum', batch_size=batch_size, device=device, 
          **extra_kwargs)

snn.set_layers()
snn.to(device)
print(snn)

num_epochs = 10
lr = 1e-3
train(snn, train_loader, test_loader, lr, num_epochs, test_behavior=tb_minimal, scheduler=(10, 0.95), test_every=1)
